In [50]:
import os
import fnmatch
import pandas as pd
import numpy as np
from datetime import datetime
import shutil
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

PROD_CONCAT = "ProdConcat"
COSTO_COMPRA = "Costo Compra"
CANTIDAD_COMPRA = "Cantidad Compra"
PRECIO_VENTA = "Precio Venta"
UTILIDAD = "UTILIDAD_%"
IVA = .16
CIEN = 100

CLASIFICACION = "CLASIFICACION"
N1 = "N1"
N2 = "N2"
N3 = "N3"

def unir_dataframes(lista_df):
    """
    Une una lista de DataFrames en uno solo, validando que todos tengan las mismas columnas.

    :param lista_df: Lista de DataFrames a unir.
    :return: DataFrame combinado si todos los DataFrames tienen las mismas columnas.
    :raises ValueError: Si los DataFrames no tienen las mismas columnas.
    """
    # Validar que la lista no esté vacía
    if not lista_df:
        raise ValueError("La lista de DataFrames está vacía.")
    
    # Obtener las columnas del primer DataFrame como referencia
    columnas_referencia = lista_df[0].columns
    
    # Verificar que todos los DataFrames tengan las mismas columnas
    for i, df in enumerate(lista_df):
        if not df.columns.equals(columnas_referencia):
            raise ValueError(f"El DataFrame en la posición {i} no tiene las mismas columnas.")
    
    # Concatenar los DataFrames si pasan la validación
    df_resultado = pd.concat(lista_df, ignore_index=True)
    return df_resultado

def generar_excel_by_dataframe(df, nombre_base):
    """
    Genera un archivo Excel a partir de un DataFrame, añadiendo la fecha y hora actual al nombre del archivo.
    :param df: DataFrame a exportar.
    :param nombre_base: Nombre base del archivo (sin extensión).
    :return: Ruta completa del archivo generado.
    """
    try:
        # Obtener la fecha y hora actuales en formato 'YYYYMMDD_HHMMSS'
        fecha_hora = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
        
        # Construir el nombre del archivo
        nombre_archivo = f"{nombre_base}_{fecha_hora}.xlsx"
        
        # Exportar el DataFrame a Excel
        df.to_excel(nombre_archivo, index=False, engine="openpyxl")
        
        print(f"Archivo Excel generado exitosamente: {nombre_archivo}")
        return nombre_archivo
    except Exception as e:
        print(f"Error al generar el archivo Excel: {e}")
        return None

def archivos_excel_by_coincidencia(directorio: str, cadena: str):
    archivos_excel = []
    patron = f"*{cadena}*.xlsx"
    
    for archivo in os.listdir(directorio):
        if fnmatch.fnmatch(archivo, patron):
            archivos_excel.append(archivo)

    return archivos_excel
    
def filtrar_columnas_dataframe(dataframe, columnas):
    """
    Devuelve un DataFrame que contiene solo las columnas especificadas.

    :param dataframe: DataFrame de pandas del que se desea conservar las columnas.
    :param columnas: Lista de nombres de columnas a conservar.
    :return: DataFrame con solo las columnas especificadas.
    """
    try:
        # Verificar cuáles columnas existen en el DataFrame
        columnas_existentes = [col for col in columnas if col in dataframe.columns]
        columnas_no_existentes = [col for col in columnas if col not in dataframe.columns]

        if columnas_no_existentes:
            print(f"Las siguientes columnas no existen en el DataFrame: {columnas_no_existentes}")

        # Columnas filtradas
        dataframe = dataframe[columnas_existentes]
    except Exception as e:
        print(f"Se produjo un error al intentar filtrar las columnas: {e}")

    return dataframe
    
def validar_datos_numericos_dataframe(dataframe, columnas):
    """
    Reemplaza ceros en las columnas especificadas de un DataFrame con NaN.
    :param dataframe: DataFrame en el que se procesarán las columnas.
    :param columnas: Lista de nombres de columnas donde se reemplazarán los ceros por NaN.
    :return: DataFrame con los ceros reemplazados por NaN en las columnas especificadas.
    """
    try:
        # Validar que las columnas existan en el DataFrame
        columnas_validas = [col for col in columnas if col in dataframe.columns]
        # Reemplazar ceros por NaN en las columnas válidas
        dataframe[columnas_validas] = dataframe[columnas_validas].replace(0, np.nan)
        print(f"Ceros reemplazados por NaN en las columnas: {columnas_validas}")
    except Exception as e:
        print(f"Error al reemplazar ceros por NaN: {e}")
    return dataframe
    
def crear_carpeta(base_nombre_carpeta, ruta_base="."):
    """
    Crea una carpeta con un nombre que incluye la fecha y hora actual al final.
    
    :param base_nombre_carpeta: Nombre base para la carpeta.
    :param ruta_base: Ruta donde se creará la carpeta. Por defecto, en el directorio actual.
    :return: Ruta completa de la carpeta creada.
    """
    try:
        # Obtener la fecha y hora actuales en formato 'YYYY-MM-DDTHH-MM-SS'
        fecha_hora = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
        
        # Construir el nombre completo de la carpeta
        nombre_completo_carpeta = f"{base_nombre_carpeta}_{fecha_hora}"
        ruta_completa_carpeta = os.path.join(ruta_base, nombre_completo_carpeta)
        
        # Crear la carpeta
        os.makedirs(ruta_completa_carpeta, exist_ok=True)
        print(f"Carpeta creada: {ruta_completa_carpeta}")
        return ruta_completa_carpeta
    except Exception as e:
        print(f"Error al crear la carpeta: {e}")
        return None

def mover_archivos_a_carpeta(lista_archivos, carpeta_destino):
    """
    Mueve una lista de archivos a una carpeta destino.
    :param lista_archivos: Lista con las rutas de los archivos a mover.
    :param carpeta_destino: Ruta de la carpeta destino.
    :return: None
    """
    try:
        # Crear la carpeta destino si no existe
        carpeta_destino = crear_carpeta(carpeta_destino)
        for archivo in lista_archivos:
            if os.path.isfile(archivo):  # Verificar que el archivo existe
                destino = os.path.join(carpeta_destino, os.path.basename(archivo))  # Ruta destino
                shutil.move(archivo, destino)  # Mover el archivo
                print(f"Archivo movido: {archivo} -> {destino}")
            else:
                print(f"El archivo no existe: {archivo}")
    except Exception as e:
        print(f"Error al mover archivos: {e}")

def validar_archivos(lista_archivos):
    """
    Valida que una lista de archivos no contenga valores vacíos.
    :param lista_archivos: Lista de rutas de archivos.
    :return: True si todos los archivos son válidos, False si hay archivos faltantes.
    """
    # Filtrar archivos vacíos
    archivos_faltantes = [archivo for archivo in lista_archivos if archivo == ""]
    
    if archivos_faltantes:
        print("Error: Faltan archivos necesarios para el proceso del reporte.")
       
    
    print("Todos los archivos son válidos.")
    return True

def validar_listas_archivos_no_vacias(lista_de_listas):
    """
    Valida que ninguna lista dentro de una lista de listas esté vacía.
    
    :param lista_de_listas: Lista de listas de strings.
    :return: True si ninguna lista está vacía.
    :raises ValueError: Si alguna lista está vacía.
    """
    for i, lista in enumerate(lista_de_listas):
        if not lista:  # Verifica si la lista está vacía
            print("Error: Faltan archivos necesarios para el proceso del reporte.")
            sys.exit(1)  # Rompe la ejecución con un código de error
            return False
    
    return True

def crear_dataframe_from_excel(archivo: str, columnas: list = None, hoja: str = None):
    """
    Lee un archivo de Excel y crea un DataFrame filtrado con las columnas especificadas.

    :param archivo: Ruta del archivo Excel a leer.
    :param columnas: Lista de nombres de columnas que se desean extraer del archivo.
    :param hoja: Nombre de la hoja a leer. Si no se especifica, se lee la primera hoja por defecto.
    :return: DataFrame con las columnas filtradas, o None si ocurre un error.
    """
    try:
        # Inicializar el DataFrame
        df = None
        # Leer el archivo de Excel
        # Si no se especifica una hoja, se lee la primera por defecto
        if hoja is None:
            df = pd.read_excel(archivo, engine='openpyxl')
        else:
            # Si se especifica una hoja, se intenta leer esa hoja en particular
            df = pd.read_excel(archivo, engine='openpyxl', sheet_name=hoja)
        # Filtrar las columnas especificadas por el usuario
        if columnas:
            df = df[columnas]
        # Devolver el DataFrame filtrado
        return df
    # Manejo de excepciones
    except FileNotFoundError:
        # Error si el archivo no se encuentra en la ruta especificada
        print(f"El archivo {archivo} no fue encontrado.")
    except KeyError as e:
        # Error si una o más columnas no existen en el archivo
        print(f"Una o más columnas no se encuentran en el archivo: {e}")
    except ValueError:
        # Error si la hoja especificada no existe en el archivo
        print(f"La hoja '{hoja}' no existe en el archivo {archivo}.")
    except Exception as e:
        # Captura cualquier otro error no previsto
        print(f"Se produjo un error al procesar el archivo: {e}")

def crear_dataframe_unido_por_coincidencia_excel(directorio: str = "./", cadena: str = "", columnas: list = None, hoja: str = None):
    """
    Busca archivos Excel en un directorio que coincidan con una cadena en su nombre, 
    los convierte en DataFrames (opcionalmente filtrando columnas y seleccionando una hoja específica)
    y los une en un solo DataFrame.

    :param directorio: Ruta del directorio donde buscar archivos Excel (por defecto "./").
    :param cadena: Cadena de coincidencia para buscar archivos.
    :param columnas: Lista de columnas a seleccionar de cada archivo (opcional). Si no se proporciona, se cargan todas.
    :param hoja: Nombre de la hoja a leer (opcional). Si no se proporciona, se lee la primera hoja.
    :return: DataFrame unido de todos los archivos encontrados y leídos correctamente.
    :raises FileNotFoundError: Si no se encuentra ningún archivo que coincida con la búsqueda.
    :raises ValueError: Si no se pudieron leer archivos o unir DataFrames correctamente.
    """
    
    # Buscar archivos Excel en el directorio que contengan la cadena en su nombre
    archivos_coincidentes = archivos_excel_by_coincidencia(directorio, cadena)
    
    # Si no se encuentran archivos, lanzar un error
    if not archivos_coincidentes:
        raise FileNotFoundError(f"No se encontraron archivos que coincidan con '{cadena}' en {directorio}.")

    # Lista para almacenar los DataFrames generados a partir de los archivos encontrados
    lista_df = []

    # Iterar sobre cada archivo encontrado
    for archivo in archivos_coincidentes:
        ruta_completa = os.path.join(directorio, archivo)  # Crear la ruta completa del archivo
        
        try:
            # Crear un DataFrame a partir del archivo Excel, con la opción de filtrar columnas y seleccionar hoja
            df = crear_dataframe_from_excel(archivo=ruta_completa, columnas=columnas, hoja=hoja)
            lista_df.append(df)  # Añadir el DataFrame a la lista
        
        except Exception as e:
            # Capturar errores durante la lectura de los archivos
            print(f"Error al leer el archivo {archivo}: {e}")
    
    # Si hay DataFrames válidos en la lista, unirlos
    if lista_df:
        return unir_dataframes(lista_df)
    
    # Si no se pudieron leer los archivos correctamente, lanzar un error
    else:
        raise ValueError("No se pudieron leer los archivos correctamente.")

def ordenar_columnas_al_lado(df, columna_referencia, columnas_a_mover):
    """
    Ordena una lista de columnas al lado de una columna de referencia en un DataFrame.

    :param df: DataFrame a modificar.
    :param columna_referencia: Columna donde se insertarán las demás columnas al lado.
    :param columnas_a_mover: Lista de columnas que se deben mover al lado de la columna de referencia.
    :return: DataFrame con las columnas reordenadas.
    """
    try:
        # Validar que las columnas estén en el DataFrame
        columnas_faltantes = [col for col in columnas_a_mover + [columna_referencia] if col not in df.columns]
        if columnas_faltantes:
            raise ValueError(f"Las siguientes columnas no están en el DataFrame: {columnas_faltantes}")

        # Obtener la lista de columnas actuales
        columnas_actuales = df.columns.tolist()

        # Determinar la posición de la columna de referencia
        indice_referencia = columnas_actuales.index(columna_referencia)

        # Crear una nueva lista de columnas manteniendo el orden original
        nuevas_columnas = []
        for col in columnas_actuales:
            if col == columna_referencia:
                # Insertar la columna de referencia
                nuevas_columnas.append(col)
                # Insertar las columnas a mover justo después de la columna de referencia
                nuevas_columnas.extend([c for c in columnas_a_mover if c in columnas_actuales])
            elif col not in columnas_a_mover:
                # Mantener otras columnas que no se están moviendo
                nuevas_columnas.append(col)

        # Reordenar el DataFrame con las nuevas columnas
        df = df[nuevas_columnas]

        return df

    except Exception as e:
        print(f"Error al reordenar las columnas: {e}")
        return df  # Devolver el DataFrame original en caso de error



In [51]:
def obtener_dataframe_ventas():
    columnas_ventas_acc_tel = ["Almacen", "ProdConcat", "Cantidad", "PrecioVenta", N1, N2, N3]
    df_ventas_acc_tel = crear_dataframe_unido_por_coincidencia_excel(directorio="./Ventas", cadena="Analisis de Ventas", columnas=columnas_ventas_acc_tel)
    df_ventas_acc_tel = validar_datos_numericos_dataframe(df_ventas_acc_tel, ["Cantidad","PrecioVenta"])

    columnas_ventas_refacc = ["Almacén Salida Reparación", "Producto", "Cantidad", "PrecioVentaSinIva", N1, N2, N3]
    df_ventas_refacc = crear_dataframe_unido_por_coincidencia_excel(directorio="./Ventas", cadena="Refacciones_Consumidas", columnas=columnas_ventas_refacc)
    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_refacc = {
        "Almacén Salida Reparación": "Almacen",
        "Producto": "ProdConcat",
        "PrecioVentaSinIva": "PrecioVenta"
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_ventas_refacc.rename(columns=mapeo_columnas_refacc, inplace=True)
    df_ventas_refacc = validar_datos_numericos_dataframe(df_ventas_refacc, ["Cantidad","PrecioVenta"])
    # Calcular el 16% y sumarlo a la columna Costo_Compra
    df_ventas_refacc["PrecioVenta"] = df_ventas_refacc["PrecioVenta"]+(df_ventas_refacc["PrecioVenta"]*IVA)
    #Fusionamos las ventas de accesorios, telefonía y refacciones
    return unir_dataframes([df_ventas_acc_tel, df_ventas_refacc])


In [52]:
def df_ventas_pivot_almacen(df_ventas):
    # Agrupar por 'Almacen' y 'ProdConcat'
    df_ventas_agrupado = df_ventas.groupby(['Almacen', 'ProdConcat']).agg({
        'Cantidad': 'sum',                 # Sumar las cantidades
        'PrecioVenta': 'mean'              # Promediar el precio de venta
    }).reset_index()  # Resetear el índice para mantener las columnas de agrupación

    
    # Paso 2: Pivoteo por 'ProdConcat' y 'Almacen'
    pivot_ventas = df_ventas_agrupado.pivot_table(
        index="ProdConcat", 
        columns="Almacen", 
        values="Cantidad", 
        aggfunc="sum"  # Ya no debería ser necesario, pero lo dejamos por seguridad
    )

    # Paso 3: Renombrar las columnas del pivote agregando un prefijo
    pivot_ventas = pivot_ventas.rename(columns=lambda col: f"Ventas de {col}")

    # Paso 4: Convertir el índice del pivote a una columna para un DataFrame plano
    return pivot_ventas.reset_index()



In [53]:
def df_ventas_totales(df_ventas):
    df_ventas = filtrar_columnas_dataframe(df_ventas, [PROD_CONCAT, "Cantidad", "PrecioVenta", N1, N2, N3])
     # Agrupar por 'Almacen' y 'ProdConcat'
    df_ventas_agrupado = df_ventas.groupby(['ProdConcat']).agg({
        'Cantidad': 'sum',                 # Sumar las cantidades
        'PrecioVenta': 'mean',
        N1: 'first',
        N2: 'first',
        N3: 'first'
    }).reset_index()  # Resetear el índice para mantener las columnas de agrupación

    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_ventas = {
        "Cantidad": "Venta Global",
        "PrecioVenta": PRECIO_VENTA
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_ventas_agrupado.rename(columns=mapeo_columnas_ventas, inplace=True)

    return df_ventas_agrupado

In [54]:
def obtener_dataframe_compras():
    columnas_compras = ["Almacen", "Producto", "Costo", "Cantidad"]
    df_compras = crear_dataframe_unido_por_coincidencia_excel(directorio="./Compras", cadena="Excel_Movimientos", columnas=columnas_compras, hoja="Detalle de movimientos")
    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_compras = {
        "Producto": PROD_CONCAT,
        "Costo": COSTO_COMPRA,
        "Cantidad": CANTIDAD_COMPRA
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_compras.rename(columns=mapeo_columnas_compras, inplace=True)
    df_compras = validar_datos_numericos_dataframe(df_compras, [COSTO_COMPRA, CANTIDAD_COMPRA])
    # Calcular el 16% y sumarlo a la columna Costo_Compra
    df_compras[COSTO_COMPRA] = df_compras[COSTO_COMPRA]+(df_compras[COSTO_COMPRA]*IVA)
    #Fusionamos las ventas de accesorios, telefonía y refacciones
    return df_compras


In [55]:
def df_compras_totales(df_compras):
    df_compras = filtrar_columnas_dataframe(df_compras, [PROD_CONCAT, COSTO_COMPRA, CANTIDAD_COMPRA])
    df_compras = df_compras.groupby([PROD_CONCAT]).agg({
        CANTIDAD_COMPRA: 'sum',                 # Sumar las cantidades
        COSTO_COMPRA: 'mean', #Promediar costos
    }).reset_index()  # Resetear el índice para mantener las columnas de agrupación
    return df_compras


In [56]:
def filtrar_ventas_by_n1_y_utilidad(df, n1, registros, group_n= N2,  asc_ventas=False, asc_utilidad=True):
    """
    Filtra el DataFrame para devolver los n registros con las mayores ventas globales
    y menor utilidad, pero solo dentro del subconjunto filtrado por N1.

    :param df: DataFrame con las columnas 'Venta Global', 'UTILIDAD', 'N1' y 'N2'.
    :param n1: Valor de la columna N1 para filtrar los datos.
    :param n: Número de registros a devolver por cada valor único de N2.
    :return: DataFrame con los registros filtrados.
    """
    try:
        # Validar que el DataFrame tenga las columnas necesarias
        columnas_requeridas = ['Venta Global', UTILIDAD, N1, N2]
        for columna in columnas_requeridas:
            if columna not in df.columns:
                raise ValueError(f"El DataFrame no contiene la columna requerida: {columna}")

        # Filtrar el DataFrame solo para los registros que coincidan con el valor de N1
        df_filtrado_n1 = df[df[N1] == n1]

        # Verificar si hay registros después del filtrado
        if df_filtrado_n1.empty:
            print(f"No se encontraron registros para N1 = {n1}")
            return pd.DataFrame()

        # Ordenar el DataFrame por 'Venta Global' (descendente) y 'UTILIDAD' (ascendente)
        df_ordenado = df_filtrado_n1.sort_values(by=['Venta Global', UTILIDAD], ascending=[asc_ventas, asc_utilidad])

        # Tomar los primeros n registros por cada valor único de N2
        df_filtrado_final = df_ordenado.groupby(group_n).head(registros)

        return df_filtrado_final

    except Exception as e:
        print(f"Error al filtrar ventas con baja utilidad: {e}")
        return pd.DataFrame()  # Retornar DataFrame vacío en caso de error



In [57]:
def generar_grafica_ventas_utilidad_por_categoria(df, columna_categoria='N2', nombre_pdf="reporte_ventas_utilidad.pdf"):
    """
    Genera gráficas de barras separadas por la categoría especificada ('N2' o 'N3'),
    mostrando 'ProdConcat' en el eje X y 'Venta Global' en el eje Y. Se añade 'UTILIDAD_%'
    como etiqueta dentro de cada barra. Se ajusta el tamaño de los labels del eje X.

    :param df: DataFrame que contiene las columnas 'ProdConcat', 'Venta Global', 'UTILIDAD_%' y una columna de categoría (N2 o N3).
    :param columna_categoria: Columna por la que se segmentarán las gráficas ('N2' o 'N3').
    :param nombre_pdf: Nombre del archivo PDF de salida.
    """
    try:
        # Validar que las columnas necesarias existan
        columnas_requeridas = [PROD_CONCAT, 'Venta Global', UTILIDAD, columna_categoria]
        for columna in columnas_requeridas:
            if columna not in df.columns:
                raise ValueError(f"El DataFrame no contiene la columna requerida: {columna}")

        # Crear el archivo PDF para almacenar las gráficas
        with PdfPages(nombre_pdf) as pdf:
            # Obtener los valores únicos de la columna de categoría (N2 o N3)
            categorias_unicas = df[columna_categoria].unique()

            # Generar una gráfica por cada valor único de la columna de categoría
            for categoria in categorias_unicas:
                # Filtrar el DataFrame por la categoría actual
                df_categoria = df[df[columna_categoria] == categoria]

                # Ordenar por ventas globales para mejor visualización
                df_categoria = df_categoria.sort_values(by='Venta Global', ascending=False)

                # Crear una figura para la gráfica
                plt.figure(figsize=(10, 6))
                barras = plt.bar(df_categoria[PROD_CONCAT], df_categoria['Venta Global'], color='skyblue')

                # Añadir etiquetas de utilidad dentro de las barras
                for barra, utilidad in zip(barras, df_categoria[UTILIDAD]):
                    plt.text(
                        barra.get_x() + barra.get_width() / 2,
                        barra.get_height() / 2,
                        f"Utilidad={utilidad:.2f}%",
                        ha='center',
                        va='center',
                        fontsize=6,
                        color='black'
                    )

                # Configuración de etiquetas y título
                plt.title(f"Ventas Globales - Categoría: {categoria}", fontsize=14)
                plt.xlabel("Producto")
                plt.ylabel("Venta Global")

                # Obtener el texto hasta el primer ' -' de cada ProdConcat
                labels_reducidos = df_categoria[PROD_CONCAT].str.split(' -').str[0]

                # Ajustar el tamaño de los labels del eje X y rotarlos
                plt.xticks(rotation=45, ha='right', fontsize=8)  # Reducir tamaño de letra
                plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños

                plt.tight_layout()

                # Guardar la gráfica en el PDF
                pdf.savefig()
                plt.close()

            print(f"Gráficas generadas y guardadas en: {nombre_pdf}")

    except Exception as e:
        print(f"Error al generar las gráficas: {e}")


In [58]:


def generar_grafica_utilidad_con_ventas_por_categoria(df, columna_categoria='N2', nombre_pdf="reporte_utilidad_ventas.pdf"):
    """
    Genera gráficas de barras separadas por la categoría especificada ('N2' o 'N3'),
    mostrando 'ProdConcat' en el eje X y 'UTILIDAD_%' en el eje Y. Se añade 'Venta Global'
    como etiqueta dentro de cada barra. Se ajusta el tamaño de los labels del eje X.

    :param df: DataFrame que contiene las columnas 'ProdConcat', 'Venta Global', 'UTILIDAD_%' y una columna de categoría (N2 o N3).
    :param columna_categoria: Columna por la que se segmentarán las gráficas ('N2' o 'N3').
    :param nombre_pdf: Nombre del archivo PDF de salida.
    """
    try:
        # Validar que las columnas necesarias existan
        columnas_requeridas = [PROD_CONCAT, 'Venta Global', UTILIDAD, columna_categoria]
        for columna in columnas_requeridas:
            if columna not in df.columns:
                raise ValueError(f"El DataFrame no contiene la columna requerida: {columna}")

        # Crear el archivo PDF para almacenar las gráficas
        with PdfPages(nombre_pdf) as pdf:
            # Obtener los valores únicos de la columna de categoría (N2 o N3)
            categorias_unicas = df[columna_categoria].unique()

            # Generar una gráfica por cada valor único de la columna de categoría
            for categoria in categorias_unicas:
                # Filtrar el DataFrame por la categoría actual
                df_categoria = df[df[columna_categoria] == categoria]

                # Ordenar por utilidad (%) para priorizar productos con menor utilidad
                df_categoria = df_categoria.sort_values(by=UTILIDAD, ascending=False)

                # Crear una figura para la gráfica
                plt.figure(figsize=(10, 6))
                barras = plt.bar(df_categoria[PROD_CONCAT], df_categoria[UTILIDAD], color='orange')

                # Añadir etiquetas de venta global dentro de las barras
                for barra, venta in zip(barras, df_categoria['Venta Global']):
                    plt.text(
                        barra.get_x() + barra.get_width() / 2,
                        barra.get_height() / 2,
                        f"Unidades={venta:,.2f}",  # Muestra venta global con formato de miles
                        ha='center',
                        va='center',
                        fontsize=6,
                        color='black'
                    )

                # Configuración de etiquetas y título
                plt.title(f"Utilidad % - Categoría: {categoria}", fontsize=14)
                plt.xlabel("Producto")
                plt.ylabel("Utilidad %")

                # Reducir el texto de 'ProdConcat' tomando solo hasta el primer ' -'
                labels_reducidos = df_categoria[PROD_CONCAT].str.split(' -').str[0]

                # Ajustar el tamaño de los labels del eje X y rotarlos
                plt.xticks(rotation=45, ha='right', fontsize=8)
                plt.gca().set_xticklabels(labels_reducidos, fontsize=7)

                plt.tight_layout()

                # Guardar la gráfica en el PDF
                pdf.savefig()
                plt.close()

            print(f"Gráficas generadas y guardadas en: {nombre_pdf}")

    except Exception as e:
        # Manejo de errores y notificación del problema
        print(f"Error al generar las gráficas: {e}")


In [59]:
#INICIO DE PROGRAMA

#Dataframes iniciales (No los mutamos para poder usarlos)
df_ventas = obtener_dataframe_ventas()
df_compras = obtener_dataframe_compras()

#Dataframe que muestra la cantidad de ventas por almacen como columnas para cada producto
df_ventas_pivote_almacenes = df_ventas_pivot_almacen(df_ventas.copy(deep=True))

#Dataframe que contiene la sumatoria de las ventas de un producto sin importar el almacen
df_ventas_totales = df_ventas_totales(df_ventas.copy(deep=True))

# Realizar el merge entre df_ventas_pivote_almacenes y df_ventas_totales por ProdConcat
df_ventas_final = pd.merge(
    df_ventas_pivote_almacenes,  # DataFrame de ventas pivoteadas por almacén
    df_ventas_totales,           # DataFrame de ventas totales
    on=PROD_CONCAT,             # Clave de unión (ProdConcat)
    how="inner"                  # Tipo de merge (inner join)
)

#Dataframe que tiene la sumatoria de las compras por productos sin importar el almacen
df_compras_agrupadas = df_compras_totales(df_compras.copy(deep=True))

# Realizar el merge entre df_ventas_final y df_compras_agrupadas por ProdConcat
df_compras_ventas = pd.merge(
    df_ventas_final,  # DataFrame de ventas pivoteadas por almacén
    df_compras_agrupadas,           # DataFrame de ventas totales
    on=PROD_CONCAT,             # Clave de unión (ProdConcat)
    how="inner"                  # Tipo de merge (inner join)
)

# Calcular la utilidad en porcentaje
df_compras_ventas[UTILIDAD] = ((df_compras_ventas[PRECIO_VENTA] - df_compras_ventas[COSTO_COMPRA]) / df_compras_ventas[COSTO_COMPRA]) * CIEN

generar_excel_by_dataframe(df_compras_ventas, "BI-DATA-TRATADA-SIN-FILTROS")

df_compras_ventas = ordenar_columnas_al_lado(df_compras_ventas, PROD_CONCAT, [N1,N2,N3])
TOP_REGISTROS= 8
df_max_ventas_acc = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="INNOVACION MOVIL", registros=TOP_REGISTROS, group_n=N3, asc_ventas=False, asc_utilidad=True)
df_min_utilidad_acc = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="INNOVACION MOVIL", registros=TOP_REGISTROS, group_n=N3, asc_ventas=True, asc_utilidad=True)
df_max_ventas_tel = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="TECNOLOGIA MOVIL", registros=TOP_REGISTROS, group_n=N2, asc_ventas=False, asc_utilidad=True)
df_min_utilidad_tel = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="TECNOLOGIA MOVIL", registros=TOP_REGISTROS, group_n=N2, asc_ventas=True, asc_utilidad=True)
df_max_ventas_refacc = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="SOLUCIONES TECNICAS", registros=TOP_REGISTROS, group_n=N2, asc_ventas=False, asc_utilidad=True)
df_min_utilidad_refacc = filtrar_ventas_by_n1_y_utilidad(df_compras_ventas.copy(deep=True), n1="SOLUCIONES TECNICAS", registros=TOP_REGISTROS, group_n=N2, asc_ventas=True, asc_utilidad=True)

generar_grafica_ventas_utilidad_por_categoria(df=df_max_ventas_acc, columna_categoria=N3, nombre_pdf="BI-MAX-VENTAS-ACCESORIOS.pdf")
generar_grafica_utilidad_con_ventas_por_categoria(df=df_min_utilidad_acc, columna_categoria=N3, nombre_pdf="BI-MIN-UTILIDAD-ACCESORIOS.pdf")
generar_grafica_ventas_utilidad_por_categoria(df=df_max_ventas_tel, columna_categoria=N2, nombre_pdf="BI-MAX-VENTAS-TELEFONIA.pdf")
generar_grafica_utilidad_con_ventas_por_categoria(df=df_min_utilidad_tel, columna_categoria=N2, nombre_pdf="BI-MIN-UTILIDAD-TELEFONIA.pdf")
generar_grafica_ventas_utilidad_por_categoria(df=df_max_ventas_refacc, columna_categoria=N2, nombre_pdf="BI-MAX-VENTAS-REFACCIONES.pdf")
generar_grafica_utilidad_con_ventas_por_categoria(df=df_min_utilidad_refacc, columna_categoria=N2, nombre_pdf="BI-MIN-UTILIDAD-REFACCIONES.pdf")











Ceros reemplazados por NaN en las columnas: ['Cantidad', 'PrecioVenta']
Ceros reemplazados por NaN en las columnas: ['Cantidad', 'PrecioVenta']
Ceros reemplazados por NaN en las columnas: ['Costo Compra', 'Cantidad Compra']
Archivo Excel generado exitosamente: BI-DATA-TRATADA-SIN-FILTROS_2024-12-23T22-58-30.xlsx


C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after se

Gráficas generadas y guardadas en: BI-MAX-VENTAS-ACCESORIOS.pdf


C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, 

Gráficas generadas y guardadas en: BI-MIN-UTILIDAD-ACCESORIOS.pdf


...

Gráficas generadas y guardadas en: BI-MIN-UTILIDAD-TELEFONIA.pdf


C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\2460073695.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)  # Labels aún más pequeños


Gráficas generadas y guardadas en: BI-MAX-VENTAS-REFACCIONES.pdf


C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)
C:\Users\gibra\AppData\Local\Temp\ipykernel_3780\3079940885.py:57: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  plt.gca().set_xticklabels(labels_reducidos, fontsize=7)


Gráficas generadas y guardadas en: BI-MIN-UTILIDAD-REFACCIONES.pdf
